# プロンプトキャッシング 101

OpenAIは1024トークンを超えるプロンプトに対して割引価格のプロンプトキャッシュを提供しており、10,000トークンを超える長いプロンプトでは最大80%のレイテンシ削減を実現します。LLM APIリクエスト間で繰り返し情報をキャッシュすることで、レイテンシとコストの両方を大幅に削減できます。プロンプトキャッシュは組織レベルでスコープされており、同じ組織のメンバーのみが共有キャッシュにアクセスできます。また、プロセス中にデータが保存されないため、キャッシュはゼロデータ保持の対象となります。

プロンプトキャッシュは1024トークンより長いプロンプトに対して自動的に有効になります。completionsリクエストで何も変更する必要はありません。APIリクエストが行われると、システムはまずプロンプトの開始部分（プレフィックス）が既にキャッシュされているかどうかを確認します。一致が見つかった場合（キャッシュヒット）、キャッシュされたプロンプトが使用され、レイテンシとコストが削減されます。一致しない場合、システムは完全なプロンプトを最初から処理し、将来の使用のためにプレフィックスをキャッシュします。

これらの利点を踏まえて、プロンプトキャッシュが特に有利となる主要なユースケースは以下の通りです：

- **ツールと構造化出力を使用するエージェント**: 拡張されたツールとスキーマのリストをキャッシュします。
- **コーディングと文章作成アシスタント**: コードベースやワークスペースの大きなセクションや要約をプロンプトに直接挿入します。
- **チャットボット**: マルチターン会話の静的部分をキャッシュして、長時間の対話にわたって効率的にコンテキストを維持します。

このクックブックでは、ツールと画像のキャッシュの例をいくつか紹介します。一般的に、指示や例などの静的コンテンツはプロンプトの冒頭に配置し、ユーザー固有の情報などの可変コンテンツは末尾に配置することが推奨されることを思い出してください。これは画像やツールにも適用され、リクエスト間で順序も含めて同一である必要があります。1024トークン未満のリクエストを含むすべてのリクエストでは、`usage.prompt_tokens_details`チャット補完オブジェクトの`cached_tokens`フィールドが表示され、プロンプトトークンのうち何個がキャッシュヒットだったかを示します。1024トークン未満のリクエストでは、`cached_tokens`はゼロになります。キャッシュ割引は、画像に使用されるトークンを含む実際に処理されたトークン数に基づいており、これらもレート制限にカウントされます。

## 例1: ツールのキャッシュと複数ターンの会話

この例では、配送日の確認、注文のキャンセル、支払い方法の更新などのタスクを処理できるカスタマーサポートアシスタント用のツールとインタラクションを定義します。アシスタントは2つの別々のメッセージを処理し、最初に初期クエリに応答し、その後フォローアップクエリに対して遅延応答を行います。

ツールをキャッシュする際は、ツール定義とその順序がプロンプトプレフィックスに含まれるために同一である必要があることが重要です。複数ターンの会話でメッセージ履歴をキャッシュするには、messagesの配列の末尾に新しい要素を追加します。レスポンスオブジェクトと以下の出力で、2番目の完了`run2`において、`cached_tokens`の値がゼロより大きいことが確認でき、これはキャッシュが成功していることを示しています。

In [4]:
from openai import OpenAI
import os
import json 
import time


api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(organization='org-l89177bnhkme4a44292n5r3j', api_key=api_key)

In [5]:
import time
import json

# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_order",
            "description": "Cancel an order that has not yet been shipped. Use this when a customer requests order cancellation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "reason": {
                        "type": "string",
                        "description": "The reason for cancelling the order."
                    }
                },
                "required": ["order_id", "reason"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "return_item",
            "description": "Process a return for an order. This should be called when a customer wants to return an item and the order has already been delivered.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "item_id": {
                        "type": "string",
                        "description": "The specific item ID the customer wants to return."
                    },
                    "reason": {
                        "type": "string",
                        "description": "The reason for returning the item."
                    }
                },
                "required": ["order_id", "item_id", "reason"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "update_shipping_address",
            "description": "Update the shipping address for an order that hasn't been shipped yet. Use this if the customer wants to change their delivery address.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "new_address": {
                        "type": "object",
                        "properties": {
                            "street": {
                                "type": "string",
                                "description": "The new street address."
                            },
                            "city": {
                                "type": "string",
                                "description": "The new city."
                            },
                            "state": {
                                "type": "string",
                                "description": "The new state."
                            },
                            "zip": {
                                "type": "string",
                                "description": "The new zip code."
                            },
                            "country": {
                                "type": "string",
                                "description": "The new country."
                            }
                        },
                        "required": ["street", "city", "state", "zip", "country"],
                        "additionalProperties": False
                    }
                },
                "required": ["order_id", "new_address"],
                "additionalProperties": False
            }
        }
    },
    # New tool: Update payment method
    {
        "type": "function",
        "function": {
            "name": "update_payment_method",
            "description": "Update the payment method for an order that hasn't been completed yet. Use this if the customer wants to change their payment details.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "payment_method": {
                        "type": "object",
                        "properties": {
                            "card_number": {
                                "type": "string",
                                "description": "The new credit card number."
                            },
                            "expiry_date": {
                                "type": "string",
                                "description": "The new credit card expiry date in MM/YY format."
                            },
                            "cvv": {
                                "type": "string",
                                "description": "The new credit card CVV code."
                            }
                        },
                        "required": ["card_number", "expiry_date", "cvv"],
                        "additionalProperties": False
                    }
                },
                "required": ["order_id", "payment_method"],
                "additionalProperties": False
            }
        }
    }
]

# Enhanced system message with guardrails
messages = [
    {
        "role": "system", 
        "content": (
            "You are a professional, empathetic, and efficient customer support assistant. Your mission is to provide fast, clear, "
            "and comprehensive assistance to customers while maintaining a warm and approachable tone. "
            "Always express empathy, especially when the user seems frustrated or concerned, and ensure that your language is polite and professional. "
            "Use simple and clear communication to avoid any misunderstanding, and confirm actions with the user before proceeding. "
            "In more complex or time-sensitive cases, assure the user that you're taking swift action and provide regular updates. "
            "Adapt to the user’s tone: remain calm, friendly, and understanding, even in stressful or difficult situations."
            "\n\n"
            "Additionally, there are several important guardrails that you must adhere to while assisting users:"
            "\n\n"
            "1. **Confidentiality and Data Privacy**: Do not share any sensitive information about the company or other users. When handling personal details such as order IDs, addresses, or payment methods, ensure that the information is treated with the highest confidentiality. If a user requests access to their data, only provide the necessary information relevant to their request, ensuring no other user's information is accidentally revealed."
            "\n\n"
            "2. **Secure Payment Handling**: When updating payment details or processing refunds, always ensure that payment data such as credit card numbers, CVVs, and expiration dates are transmitted and stored securely. Never display or log full credit card numbers. Confirm with the user before processing any payment changes or refunds."
            "\n\n"
            "3. **Respect Boundaries**: If a user expresses frustration or dissatisfaction, remain calm and empathetic but avoid overstepping professional boundaries. Do not make personal judgments, and refrain from using language that might escalate the situation. Stick to factual information and clear solutions to resolve the user's concerns."
            "\n\n"
            "4. **Legal Compliance**: Ensure that all actions you take comply with legal and regulatory standards. For example, if the user requests a refund, cancellation, or return, follow the company’s refund policies strictly. If the order cannot be canceled due to being shipped or another restriction, explain the policy clearly but sympathetically."
            "\n\n"
            "5. **Consistency**: Always provide consistent information that aligns with company policies. If unsure about a company policy, communicate clearly with the user, letting them know that you are verifying the information, and avoid providing false promises. If escalating an issue to another team, inform the user and provide a realistic timeline for when they can expect a resolution."
            "\n\n"
            "6. **User Empowerment**: Whenever possible, empower the user to make informed decisions. Provide them with relevant options and explain each clearly, ensuring that they understand the consequences of each choice (e.g., canceling an order may result in loss of loyalty points, etc.). Ensure that your assistance supports their autonomy."
            "\n\n"
            "7. **No Speculative Information**: Do not speculate about outcomes or provide information that you are not certain of. Always stick to verified facts when discussing order statuses, policies, or potential resolutions. If something is unclear, tell the user you will investigate further before making any commitments."
            "\n\n"
            "8. **Respectful and Inclusive Language**: Ensure that your language remains inclusive and respectful, regardless of the user’s tone. Avoid making assumptions based on limited information and be mindful of diverse user needs and backgrounds."
        )
    },
    {
        "role": "user", 
        "content": (
            "Hi, I placed an order three days ago and haven’t received any updates on when it’s going to be delivered. "
            "Could you help me check the delivery date? My order number is #9876543210. I’m a little worried because I need this item urgently."
        )
    }
]

# Enhanced user_query2
user_query2 = {
    "role": "user", 
    "content": (
        "Since my order hasn't actually shipped yet, I would like to cancel it. "
        "The order number is #9876543210, and I need to cancel because I’ve decided to purchase it locally to get it faster. "
        "Can you help me with that? Thank you!"
    )
}

# Function to run completion with the provided message history and tools
def completion_run(messages, tools):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        tools=tools,
        messages=messages,
        tool_choice="required"
    )
    usage_data = json.dumps(completion.to_dict(), indent=4)
    return usage_data

# Main function to handle the two runs
def main(messages, tools, user_query2):
    # Run 1: Initial query
    print("Run 1:")
    run1 = completion_run(messages, tools)
    print(run1)

    # Delay for 7 seconds
    time.sleep(7)

    # Append user_query2 to the message history
    messages.append(user_query2)

    # Run 2: With appended query
    print("\nRun 2:")
    run2 = completion_run(messages, tools)
    print(run2)


# Run the main function
main(messages, tools, user_query2)


Run 1:
{
    "id": "chatcmpl-ADeOueQSi2DIUMdLXnZIv9caVfnro",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_5TnLcdD9tyVMVbzNGdejlJJa",
                        "function": {
                            "arguments": "{\"order_id\":\"9876543210\"}",
                            "name": "get_delivery_date"
                        },
                        "type": "function"
                    }
                ]
            }
        }
    ],
    "created": 1727816928,
    "model": "gpt-4o-mini-2024-07-18",
    "object": "chat.completion",
    "system_fingerprint": "fp_f85bea6784",
    "usage": {
        "completion_tokens": 17,
        "prompt_tokens": 1079,
        "total_tokens": 1096,
        "prompt_

## 例2: 画像

2番目の例では、ユーザークエリと共に、食料品アイテムの複数の画像URLをmessages配列に含め、遅延を設けて3回実行します。画像は、リンクされたものでもユーザーメッセージ内でbase64エンコードされたものでも、キャッシュの対象となります。`detail`パラメータは画像のトークン化に影響するため、一貫性を保つようにしてください。GPT-4o-miniはテキストに低コストのトークンモデルを使用していますが、画像処理コストをカバーするために追加のトークンを加算することに注意してください。キャッシュ割引は、画像に使用されるトークンを含む実際に処理されたトークン数に基づいており、これらもレート制限にカウントされます。

この例の出力では、2回目の実行でキャッシュがヒットしたことが示されていますが、ユーザークエリは同じであっても、最初のURLが異なる（`veggie_url`ではなく`eggs_url`）ため、3回目の実行ではキャッシュがヒットしませんでした。

In [8]:
sauce_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/12-04-20-saucen-by-RalfR-15.jpg/800px-12-04-20-saucen-by-RalfR-15.jpg"
veggie_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Veggies.jpg/800px-Veggies.jpg"
eggs_url= "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Egg_shelf.jpg/450px-Egg_shelf.jpg"
milk_url= "https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Lactaid_brand.jpg/800px-Lactaid_brand.jpg"

def multiimage_completion(url1, url2, user_query):
    completion = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {
        "role": "user",
        "content": [
            {
            "type": "image_url",
            "image_url": {
                "url": url1,
                "detail": "high"
            },
            },
            {
            "type": "image_url",
            "image_url": {
                "url": url2,
                "detail": "high"
            },
            },
            {"type": "text", "text": user_query}
        ],
        }
    ],
    max_tokens=300,
    )
    print(json.dumps(completion.to_dict(), indent=4))
    

def main(sauce_url, veggie_url):
    multiimage_completion(sauce_url, veggie_url, "Please list the types of sauces are shown in these images")
    #delay for 20 seconds
    time.sleep(20)
    multiimage_completion(sauce_url, veggie_url, "Please list the types of vegetables are shown in these images")
    time.sleep(20)
    multiimage_completion(milk_url, sauce_url, "Please list the types of sauces are shown in these images")

if __name__ == "__main__":
    main(sauce_url, veggie_url)

{
    "id": "chatcmpl-ADeV3IrUqhpjMXEgv29BFHtTQ0Pzt",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "The images show the following types of sauces:\n\n1. **Soy Sauce** - Kikkoman brand.\n2. **Worcester Sauce** - Appel brand, listed as \"Dresdner Art.\"\n3. **Tabasco Sauce** - Original pepper sauce.\n\nThe second image shows various vegetables, not sauces.",
                "refusal": null,
                "role": "assistant"
            }
        }
    ],
    "created": 1727817309,
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "system_fingerprint": "fp_2f406b9113",
    "usage": {
        "completion_tokens": 65,
        "prompt_tokens": 1548,
        "total_tokens": 1613,
        "prompt_tokens_details": {
            "cached_tokens": 0
        },
        "completion_tokens_details": {
            "reasoning_tokens": 0
        }
    }
}
{


## 全体的なヒント

プロンプトキャッシュを最大限に活用するために、以下のベストプラクティスに従うことを検討してください：

- 静的または頻繁に再利用されるコンテンツをプロンプトの冒頭に配置する：動的データをプロンプトの末尾に向けて配置することで、より良いキャッシュ効率を確保できます。

- 一貫した使用パターンを維持する：定期的に使用されないプロンプトは自動的にキャッシュから削除されます。キャッシュの削除を防ぐために、プロンプトの一貫した使用を維持してください。

- 主要な指標を監視する：キャッシュヒット率、レイテンシ、キャッシュされたトークンの割合を定期的に追跡してください。これらの洞察を使用してキャッシュ戦略を微調整し、パフォーマンスを最大化してください。

これらの実践を実装することで、プロンプトキャッシュを最大限に活用し、アプリケーションが応答性とコスト効率の両方を確保できます。適切に管理されたキャッシュ戦略により、処理時間を大幅に短縮し、コストを削減し、スムーズなユーザーエクスペリエンスの維持に役立ちます。